In [14]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.Encoder
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.TypedColumn
import org.apache.spark.sql.functions._
import org.apache.spark.sql.SQLContext
val sqlContext = SQLContext.getOrCreate(sc)
import sqlContext.implicits._

In [80]:
val paidAgg = new Aggregator[(Int, String, Double), (Double, Boolean), Option[Double]] with Serializable {
	/* The initial value */
	def zero: (Double, Boolean) = (0.0, false)
	
	/* Add an element to the running total */
	def reduce(b: (Double, Boolean), a: (Int, String, Double)) = 
	{
		val inc = if (a._2 == "PAID") a._3 else 0.0
		val isCheckedIn = (a._2 == "Checked In")
		(b._1 + inc, b._2 || isCheckedIn)
	}

	/* Merge intermediate values */
	def merge(b1: (Double, Boolean), b2: (Double, Boolean)) = (b1._1 + b2._1, b1._2 || b2._2)
	
	/* Return the final result */
	def finish(b: (Double, Boolean)) = if (b._2) Some(b._1) else None
}.toColumn

In [22]:
val df = Seq((10, "Checked out", 1.0), (10, "PAID", 40.0), (10, "Checked In", 1.0), (15, "Flew Away", 2.0),
             (15, "PAID", 100.00), (15, "Came back", 1.0), (20, "PAID", 150.00), (30, "Checked In", 1.0), 
             (30, "PAID", 50.00), (30, "PAID", 10.00)).toDF("CustNum", "Action", "Val")

val df2 = Seq((10, ("Checked out", 1.0)), (10, ("PAID", 40.0))).toDF("CustNum", "Action-Val")

In [23]:
df2.show

+-------+-----------------+
|CustNum|       Action-Val|
+-------+-----------------+
|     10|[Checked out,1.0]|
|     10|      [PAID,40.0]|
+-------+-----------------+



In [27]:
print(df2.schema)

StructType(StructField(CustNum,IntegerType,false), StructField(Action-Val,StructType(StructField(_1,StringType,true), StructField(_2,DoubleType,false)),true))

In [81]:
df.as[(Int, String, Double)].groupBy(_._1).agg(paidAgg).show

+---+----------+
| _1|        _2|
+---+----------+
| 10|Some(40.0)|
| 15|      None|
| 20|      None|
| 30|Some(60.0)|
+---+----------+



In [82]:
df.groupBy($"CustNum").agg(paidAgg)

Name: org.apache.spark.sql.AnalysisException
Message: unresolved operator 'Aggregate [CustNum#24], [CustNum#24,($anon$1(),mode=Complete,isDistinct=false) AS $anon$1()#618];
StackTrace: org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.failAnalysis(CheckAnalysis.scala:38)
org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:44)
org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:203)
org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:50)
org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:121)
org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.checkAnalysis(CheckAnalysis.scala:50)
org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:44)
org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:34)
org.apache.spark.sql.DataFrame.<init>(DataFrame.scala:133)
org.apache.spark.sql.Dat